<a href="https://colab.research.google.com/github/Cropus/Minimization/blob/main/3lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №3. 
## Выполнили: Белоцерковцев, Баранов

In [ ]:
import numpy as np
import random
import math
import collections

## LU-разложение, обратные матрицы, СЛАУ, разложение в CSR, генераторы даты

In [ ]:
def generate_random_sparse_matrix(N):
    m = np.zeros((N, N))
    
    for i in range(N):
        zeroes_per_row = random.randint(round(N/2), N-1)
        other_values_per_row = N - zeroes_per_row
        m[i] = [value for value in random.sample([0] * zeroes_per_row + random.sample(range(0, 100), N - zeroes_per_row), N)]
        if m[i][i] == 0:
          m[i][i] = random.randint(1, 100)
    return m

In [ ]:
def generate_random_b(N):
  m = [value for value in random.sample(range(1, 100), N)]
  return np.array(m)

In [ ]:
def generate_gilbert_matrix(N):
  m = np.zeros((N, N))
  # m[0][0] = 1

  for i in range(0, N):
    for j in range(0, N):
      m[i][j] = 1 / (i + j + 1)
  return m

In [ ]:
def generate_random_ak_matrix(N):
  matrix = np.random.randint(-5, 0, (N, N)).astype(np.float)

  for i in range(N):
    s = matrix[i].sum() - matrix[i, i]
    s = s * (-1)
    matrix[i, i] = s
  b = matrix @ np.arange(N)

  return matrix, b

In [ ]:
def condition_number_for_matrix(matrix, k):
  t_matrix = matrix.copy()
  for i in range(len(matrix)):
    t_matrix[i][i] = t_matrix[i][i] + 10**(-k)
  return t_matrix

In [ ]:
class Matrix(object):

  def __init__(self, n):
    self.N = n
    self.data = []
    self.indexes = []
    self.indptr = [0]

  # Перевод матрицы в csr
  def csr_matrix_in(self, matrix):
    for i in range(self.N):
      amount = 0
      for j in range(self.N):
        if matrix[i][j] != 0:
          self.data.append(matrix[i][j])
          self.indexes.append(j)
          amount += 1
      self.indptr.append(amount + self.indptr[-1])

  # Получение элемента с индексами ij
  def ij(self, index_i, index_j):
    values_matching_column = list()

    for index_of_value in range(0, len(self.data)):
      if self.indexes[index_of_value] == index_j:
        for row_number in range(0, len(self.indptr)-1):
          if self.indptr[row_number] <= index_of_value < self.indptr[row_number + 1] and row_number == index_i:
            return self.data[index_of_value]

    return 0

  # Возвращает стандартный вид матрицы
  def get_normal(self):
    out = np.zeros((self.N, self.N))
    for i in range(self.N):
      for j in range(self.N):
        out[i][j] = self.ij(i, j)

    return out

In [ ]:
# Разложение в LU
def decompose_to_LU(a, n):
  LU = np.matrix(np.zeros([n, n]))

  for k in range(n):
    for j in range(k, n):
      LU[k, j] = a.ij(k, j) - LU[k, :k] * LU[:k, j]
    for i in range(k + 1, n):
      LU[i, k] = (a.ij(i, k) - LU[i, : k] * LU[: k, k]) / LU[k, k]

  return LU

# Получение L
def get_L(m):
  L = m.copy()
  for i in range(L.shape[0]):
    L[i, i] = 1
    L[i, i+1 :] = 0

  matrix = Matrix(L.shape[0])
  matrix.csr_matrix_in(np.array(L))
  return matrix

# получение U
def get_U(m):
  U = m.copy()
  for i in range(1, U.shape[0]):
    U[i, :i] = 0

  matrix = Matrix(U.shape[0])
  matrix.csr_matrix_in(np.array(U))
  return matrix

# Проверка правильности разложения
def check(a, b):
  L = a.get_normal()
  U = b.get_normal()
  print(L @ U)

# Решение системы
def solve_LU(LU, b):
  L = get_L(LU)
  U = get_U(LU)
  
  # Формирование вектора y(Ly=b)
  y = np.zeros(LU.shape[0])
  for i in range(len(y)):
    sum = 0
    for k in range(i):
      sum += L.ij(i, k) * y[k]
    y[i] = b[i] - sum

  # Формирование вектора x(Ux=y)
  #x = np.zeros(LU.shape[0])
  x = np.matrix(np.zeros([LU.shape[0], 1]))
  for i in range(1, x.shape[0] + 1):
    x[-i, 0] = (y[-i] - LU[-i, -i:] * x[-i:, 0] )/ LU[-i, -i]
    #sum = 0
    #for k in range(LU.shape[0] - i):
    #  print(k)
    #  sum += U.ij(U.N - i, U.N - k - 1) * x[-k]
    #print('----')
    #x[-i] = (y[-i] - sum)/ U.ij(U.N - i, U.N -i)

  return x

# Обращение L
def invert_L(_L):
  L = _L.get_normal()
  n = _L.N
  for k in range(1, n):
    L[k][0] *= -1

  for k in range(n):
    for i in range(k+2, n):
      for j in range(i):
        if (i-1 == j):
          L[i][j] = -L[i][j]
        else:
          L[i][j] -= L[k+1][j] * L[i][k+1]
  return L

# Обращение U
def invert_U(_U):
  U = _U.get_normal()
  n = _U.N

  for k in range(n):
    U[k][k] = 1 / U[k][k]
    for j in range(k+1, n):
      U[k][j] = U[k][j] * U[k][k]

  for k in reversed(range(1, n)):
    for j in reversed(range(k, n)):
      for i in reversed(range(k)):
        if (k == j):
          U[i][j] = -U[i][j] * U[k][j]
        else:
          U[i][j] -= U[k][j] * U[i][k]

  return U

def check_invert(LU):
  _L = invert_L(get_L(LU))
  _U = invert_U(get_U(LU))
  print(_U @ _L)

## Исследования

In [ ]:
# Размерность исследуемых матриц
n = 4

### Исследование для случайных разреженных матриц

In [ ]:
m = Matrix(n)
# matrix_to_transform = np.array([[1, 2, 3], [3, 2, 1], [2, 1, 3]])
matrix_to_transform = generate_random_sparse_matrix(n)
m.csr_matrix_in(matrix_to_transform)
LU = decompose_to_LU(m, n)
L = get_L(LU)
U = get_U(LU)
print(matrix_to_transform)
print('Матрица L: \n', L.get_normal())
print('Матрица U: \n', U.get_normal())
print('L @ U:')
check(L, U)

print('Решение системы:')
# b = np.array([[9], [7], [9]])
b = generate_random_b(n)

ans = solve_LU(LU, b)
print(ans)
check_invert(LU)
             

[[11.  0.  0. 44.]
 [60. 33.  0.  0.]
 [ 0.  0. 32. 42.]
 [ 0. 80. 60. 79.]]
Матрица L: 
 [[1.         0.         0.         0.        ]
 [5.45454545 1.         0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         2.42424242 1.875      1.        ]]
Матрица U: 
 [[  11.            0.            0.           44.        ]
 [   0.           33.            0.         -240.        ]
 [   0.            0.           32.           42.        ]
 [   0.            0.            0.          582.06818182]]
L @ U:
[[11.  0.  0. 44.]
 [60. 33.  0.  0.]
 [ 0.  0. 32. 42.]
 [ 0. 80. 60. 79.]]
Решение системы:
[[0.17210053]
 [0.17193843]
 [0.33914094]
 [0.47970214]]
[[ 3.90457225e-05  0.00000000e+00 -1.28850884e-02 -6.87204717e-03]
 [-7.09922228e-05  3.03030303e-02  2.34274335e-02  1.24946312e-02]
 [-2.98167336e-02  0.00000000e+00  2.70220804e-02 -2.25489048e-03]
 [ 2.27175113e-02  0.00000000e+00  3.22127211e-03  1.71801179e-03]]


### Исследование для матриц число обусловленности которых регулируется за счет изменения диагонального преобладания

In [ ]:
matrix_def, b = generate_random_ak_matrix(n)


for k_gen in range(7, 16):
  print("========================================")
  print("Результат для числа обсуловленности ", k_gen)
  matrix_to_transform = condition_number_for_matrix( matrix_def, k_gen )
  m = Matrix(n)

  m.csr_matrix_in(matrix_to_transform)

  LU = decompose_to_LU(m, n)
  L = get_L(LU)
  U = get_U(LU)

  print(matrix_to_transform)
  print('Матрица L: \n', L.get_normal())
  print('Матрица U: \n', U.get_normal())
  print('L @ U:')

  check(L, U)

  print('Решение системы:')
  ans = solve_LU(LU, b)
  print(ans)

  print('Решение системы без погрешности:')
  ans = np.linalg.solve(matrix_to_transform, b)
  print(ans)

Результат для числа обсуловленности  7
[[ 4.0000001 -1.        -1.        -2.       ]
 [-3.        11.0000001 -4.        -4.       ]
 [-4.        -3.         9.0000001 -2.       ]
 [-4.        -4.        -4.        12.0000001]]
Матрица L: 
 [[ 1.          0.          0.          0.        ]
 [-0.74999998  1.          0.          0.        ]
 [-0.99999998 -0.3902439   1.          0.        ]
 [-0.99999998 -0.48780487 -1.19047615  1.        ]]
Матрица U: 
 [[ 4.00000010e+00 -1.00000000e+00 -1.00000000e+00 -2.00000000e+00]
 [ 0.00000000e+00  1.02500001e+01 -4.74999998e+00 -5.49999996e+00]
 [ 0.00000000e+00  0.00000000e+00  6.14634163e+00 -6.14634136e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.04761880e-07]]
L @ U:
[[ 4.0000001 -1.        -1.        -2.       ]
 [-3.        11.0000001 -4.        -4.       ]
 [-4.        -3.         9.0000001 -2.       ]
 [-4.        -4.        -4.        12.0000001]]
Решение системы:
[[-1.04724408]
 [-0.0472441 ]
 [ 0.9527559 ]
 [ 1.95275589]

### Исследование для матриц Гильберта

In [ ]:
for size in range(8, 10):
  print("========================================")
  print("Результат для размерности ", size)
  m = Matrix(size)
  matrix_to_transform = generate_gilbert_matrix(size)
  m.csr_matrix_in(matrix_to_transform)
  LU = decompose_to_LU(m, size)
  L = get_L(LU)
  U = get_U(LU)
  print(matrix_to_transform)
  print('Матрица L: \n', L.get_normal())
  print('Матрица U: \n', U.get_normal())
  print('L @ U:')
  check(L, U)

  print('Решение системы:')
  # b = np.array([[9], [7], [9]])
  b = generate_random_b(size)
  ans = solve_LU(LU, b)
  print(ans)
  ans = np.linalg.solve(matrix_to_transform, b)
  print("Результат без погрешности")
  print(ans)

Результат для размерности  8
[[1.         0.5        0.33333333 0.25       0.2        0.16666667
  0.14285714 0.125     ]
 [0.5        0.33333333 0.25       0.2        0.16666667 0.14285714
  0.125      0.11111111]
 [0.33333333 0.25       0.2        0.16666667 0.14285714 0.125
  0.11111111 0.1       ]
 [0.25       0.2        0.16666667 0.14285714 0.125      0.11111111
  0.1        0.09090909]
 [0.2        0.16666667 0.14285714 0.125      0.11111111 0.1
  0.09090909 0.08333333]
 [0.16666667 0.14285714 0.125      0.11111111 0.1        0.09090909
  0.08333333 0.07692308]
 [0.14285714 0.125      0.11111111 0.1        0.09090909 0.08333333
  0.07692308 0.07142857]
 [0.125      0.11111111 0.1        0.09090909 0.08333333 0.07692308
  0.07142857 0.06666667]]
Матрица L: 
 [[1.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.5        1.         0.         0.         0.         0.
  0.         0.        ]
 [0.33333333 1.         1.         0.         0.       